In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 40kB/s 


In [2]:
!nvidia-smi

Tue May  4 22:01:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [4]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob




In [5]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/train'
valid_path = '/content/drive/MyDrive/valid'

In [26]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights = 'imagenet', include_top=False)

In [27]:
for layer in inception.layers:
  layer.trainable=False

In [28]:
folders = glob('/content/drive/MyDrive/train/*')

In [29]:
folders

['/content/drive/MyDrive/train/sell', '/content/drive/MyDrive/train/buy']

In [30]:
x = Flatten()(inception.output)

In [31]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=inception.input, outputs=prediction)

In [32]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_188[0][0]    
____________________________________________________________________________________________

In [33]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [38]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 14 images belonging to 2 classes.


In [39]:
test_set = train_datagen.flow_from_directory('/content/drive/MyDrive/valid',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 8 images belonging to 2 classes.


In [47]:
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 4.0802 - accuracy: 0.6429 - val_loss: 6.0933 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 1s 860ms/step - loss: 5.3152 - accuracy: 0.6429 - val_loss: 4.5175 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 1s 858ms/step - loss: 2.9823 - accuracy: 0.5714 - val_loss: 3.4896 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 1s 830ms/step - loss: 1.1835 - accuracy: 0.5714 - val_loss: 1.5160 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 1s 840ms/step - loss: 1.5676 - accuracy: 0.5000 - val_loss: 3.6733 - val_accuracy: 0.3750
Epoch 6/10
1/1 [==============================] - 1s 846ms/step - loss: 4.0947 - accuracy: 0.3571 - val_loss: 3.8710 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 1s 849ms/step - loss: 3.8686 - accuracy: 0.2857 - val_loss: 1.4431 - val_accuracy: 0.6250
Epoch 8/10
1/1 [=======

In [44]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_188[0][0]    
____________________________________________________________________________________________

In [46]:
model.save('/content/drive/MyDrive/my_model') 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/my_model/assets
